## Scrape these from pdf
- Customer Number
- Customer Name
- Vessel Name/Description
- INVOICE NO
- DATE
- Invoice Amount

In [13]:
import pdfplumber
import re
import os
import pandas as pd

In [15]:
files = os.listdir('Invoices')

In [13]:
files.index("sfwk-inv-012491-1-2020.pdf")

79

# Main script

In [ ]:
# dd = list()

In [ ]:
for file in files:
    if not file.endswith(".pdf"):
        continue
    
    cus_no = ""
    cus_name = ""
    text = ""
    inv_no = ""
    date = ""
    vessel_name = ""
    inv_amount = ""
    draft = ""
    
    #GET FULL TEXT OF INVOICE
    with pdfplumber.open("Invoices/"+file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    
    if len(text)<200:
        dd.append(["", "", "", "", "", "", "", file])
        continue
            
    print(file)
    
    
    if re.match(r"DRAFT\s+INVOICE", text):
        draft = "DRAFT"
    elif re.match(r"PROFORMA\s+INVOICE", text):
        draft = "PROFORMA"
    else:
        draft = "NORMAL"

    try:
        inv_no = re.findall(r"[A-Z]{4}-[A-Z]{3}-[0-9]{6}|[A-Z]{4}/[A-Z]{3}/[0-9]{6}\s*/[0-9]{4}", text)[0]

        date = re.findall(r"[0-9]{2}/[0-9]{2}/[0-9]{4}", text)[0]

        for line in text.split("\n"):
            if re.match(r"INVOICE\s*NO\s*:", line):
                cus_no = line.split()[0]
            if re.match(r"DATE\s*:", line):
                cus_name = line.split()[0]

        inv_amount_str = re.findall(r"Invoice\sAmount\s-->\s*([\d,]+\.\d+)?", text)[0]
        inv_amount = re.sub(r',', '', inv_amount_str)


        cus_no = re.findall(r"Page:\s*\d\s*/\s*\d+\n(\d{4,6})", text, flags=re.DOTALL)[0]
        cus_name = re.findall(r"INVOICE\s*NO\s*:\s\S+\n(.*?)DATE\s*:", text, flags=re.DOTALL)[0]

        vessel_name = re.findall(r"Vessel\s*Name/Description\s*:\s*(.*)", text)[0]
    except:
        pass
    
    dd.append([cus_no, cus_name, vessel_name, date, inv_no, inv_amount, draft, file])

In [ ]:
inv_no = re.findall(r"[A-Z]{4}-[A-Z]{3}-[0-9]{6}|[A-Z]{4}/[A-Z]{3}/[0-9]{6}\s*/[0-9]{4}", text)[0]

date = re.findall(r"[0-9]{2}/[0-9]{2}/[0-9]{4}", text)[0]

for line in text.split("\n"):
    if re.match(r"INVOICE\s*NO\s*:", line):
        cus_no = line.split()[0]
    if re.match(r"DATE\s*:", line):
        cus_name = line.split()[0]

inv_amount_str = re.findall(r"Invoice\sAmount\s-->\s*([\d,]+\.\d+)?", text)[0]
inv_amount = re.sub(r',', '', inv_amount_str)


cus_no = re.findall(r"Page:\s*\d\s*/\s*\d+\n(\d{4,6})", text, flags=re.DOTALL)[0]
cus_name = re.findall(r"INVOICE\s*NO\s*:\s\S+\n(.*?)DATE\s*:", text, flags=re.DOTALL)[0]

vessel_name = re.findall(r"Vessel\s*Name/Description\s*:\s*(.*)", text)[0]


In [20]:
df = pd.DataFrame(dd, columns=["Customer Number", "Customer Name", "Vessel Name", "Date", "Invoice number", "Invoice Amount", "Type", "File name"])

In [21]:
df

,Customer Number,Customer Name,Vessel Name,Date,Invoice number,Invoice Amount,Type,File name
0,100517,QATAR ARMED FORCES,AL DOHA -LOGISTIC SUPPORT IN TURKEY,29/11/2021,SFWK-DMY-008390,2141934.70,NORMAL,008390-Turkey.pdf
1,100517,QATAR ARMED FORCES,AL DOHA-LOGISTIC SUPPORT IN OMAN,09/02/2022,SFWK-DMY-008529,1062794.07,NORMAL,008529-Oman.pdf
2,100517,QATAR ARMED FORCES,AL DOHA- LOGISTIC SUPPORT IN DJIBOUTI,09/02/2022,SFWK-DMY-008530,624170.69,NORMAL,008530-Djibouti.pdf
3,100493,H.H THE EMIR'S PRIVATE AFFAIRS OFFICE,"YACHT ""AL ADAID""",10/01/2021,SFWK-DMY-007883,1160224.00,NORMAL,20-010892-Draft Invoice dated 20.04.2021.pdf
4,103803,AMIRI YACHTS,"YACHT ""AL ADAID""",10/01/2021,SFWK-DMY-007883,1160224.00,NORMAL,20-010892-Draft Invoice dated 29.04.2021.pdf
...,...,...,...,...,...,...,...,...
483,100936,QATAR STEEL CO.LTD.\n,REPAIR OF GRP PIPELINES,28/02/2021,SFWK/INV/012638/2021,4814.00,NORMAL,SFWK_INV_012638_Repair of GRP pipe line _Qatar...
484,100936,QATAR STEEL CO.LTD.\n,QATAR STEEL,28/02/2021,SFWK/INV/012639/2021,3816.00,NORMAL,SFWK_INV_012639_Repair of GRP pipe line _Qatar...
485,900001,SHIP MANAGEMENT - SHIPYARD INTERCO Shipyard WS\n,M.V. “HALUL-30”,28/02/2021,SFWK/INV/012641/2021,332006.00,NORMAL,SFWK_INV_012641_M.V.Halul-30 _MSM.pdf
486,900001,SHIP MANAGEMENT - SHIPYARD INTERCO Shipyard WS\n,"MOORING BOAT""AL MELAIHAT""",24/03/2021,SFWK/INV/012662/2021,16585.00,NORMAL,SFWK_INV_012662_Mooring Boat Al Melaihat_MSM.pdf


In [23]:
df.to_excel("DataFrame2.xlsx", sheet_name="Sheet1", index=False)

# Read dataframe

In [44]:
df = pd.read_excel("DataFrame.xlsx")

In [25]:
df

,Customer Number,Customer Name,Vessel Name,Date,Invoice number,Invoice Amount,Type,File name
0,100517,SHIP REPAIR & FABRICATION,AL DOHA -LOGISTIC SUPPORT IN TURKEY,29/11/2021,SFWK-DMY-008390,2141934.70,DRAFT,008390-Turkey.pdf
1,100517,SHIP REPAIR & FABRICATION,AL DOHA-LOGISTIC SUPPORT IN OMAN,09/02/2022,SFWK-DMY-008529,1062794.07,DRAFT,008529-Oman.pdf
2,100517,SHIP REPAIR & FABRICATION,AL DOHA- LOGISTIC SUPPORT IN DJIBOUTI,09/02/2022,SFWK-DMY-008530,624170.69,DRAFT,008530-Djibouti.pdf
3,100493,SHIP REPAIR & FABRICATION,"YACHT ""AL ADAID""",10/01/2021,SFWK-DMY-007883,1160224.00,DRAFT,20-010892-Draft Invoice dated 20.04.2021.pdf
4,103803,SHIP REPAIR & FABRICATION,"YACHT ""AL ADAID""",10/01/2021,SFWK-DMY-007883,1160224.00,DRAFT,20-010892-Draft Invoice dated 29.04.2021.pdf
...,...,...,...,...,...,...,...,...
40,103519,SHIP REPAIR & FABRICATION,"SURVEY BOAT""AL MASSAH""",31/08/2021,SFWK-DMY-008228,13542.50,DRAFT,Draft Invoice.pdf
41,,,,,,,,Final Invoice as per contract - MY Daloob.pdf
42,,,,,,,,Invoice -Svitzer Al Shamal.pdf
43,,,,,,,,Invoice-Svitzer Al Shamal.pdf


In [26]:
pd.set_option('display.max_rows', None)

In [27]:
df

,Customer Number,Customer Name,Vessel Name,Date,Invoice number,Invoice Amount,Type,File name
0,100517,SHIP REPAIR & FABRICATION,AL DOHA -LOGISTIC SUPPORT IN TURKEY,29/11/2021,SFWK-DMY-008390,2141934.70,DRAFT,008390-Turkey.pdf
1,100517,SHIP REPAIR & FABRICATION,AL DOHA-LOGISTIC SUPPORT IN OMAN,09/02/2022,SFWK-DMY-008529,1062794.07,DRAFT,008529-Oman.pdf
2,100517,SHIP REPAIR & FABRICATION,AL DOHA- LOGISTIC SUPPORT IN DJIBOUTI,09/02/2022,SFWK-DMY-008530,624170.69,DRAFT,008530-Djibouti.pdf
3,100493,SHIP REPAIR & FABRICATION,"YACHT ""AL ADAID""",10/01/2021,SFWK-DMY-007883,1160224.00,DRAFT,20-010892-Draft Invoice dated 20.04.2021.pdf
4,103803,SHIP REPAIR & FABRICATION,"YACHT ""AL ADAID""",10/01/2021,SFWK-DMY-007883,1160224.00,DRAFT,20-010892-Draft Invoice dated 29.04.2021.pdf
5,100493,SHIP REPAIR & FABRICATION,"YACHT ""AL ADAID""",10/01/2021,SFWK-DMY-007883,1199421.00,DRAFT,20-010892-Draft Invoice.pdf
6,103519,SHIP REPAIR & FABRICATION,"SURVEY BOAT""AL MASSAH""",31/08/2021,SFWK-DMY-008228,63396.13,DRAFT,21-011020- Al Massah-MME.pdf
7,102830,SHIP REPAIR & FABRICATION,“YACHT SAFA (MAJESTY-77)”,10/05/2021,SFWK-DMY-008061,166505.00,DRAFT,21-011043- Draft invoice.pdf
8,102830,SHIP REPAIR & FABRICATION,“YACHT SAFA (MAJESTY-77)”,10/05/2021,SFWK-DMY-008061,125285.00,PROFORMA,21-011043- Yacht Safa-Proforma Invoice for Pay...
9,100493,SHIP REPAIR & FABRICATION,YACHT YXT-ONE,13/03/2021,SFWK-DMY-007962,164430.00,DRAFT,21-011078-Yacht YXT One.pdf


In [28]:
pd.set_option('display.max_rows', 10)

# TEST SCRIPT

In [7]:
file = "sfwk-inv-012491-1-2020.pdf"

In [8]:
text = ""

with pdfplumber.open("Invoices/"+file) as pdf:
    for page in pdf.pages:
        text += page.extract_text()

# print(file)

In [9]:
print(text)

QATAR NAVIGATION (QPSC)
SHIP REPAIR & FABRICATION
Telephones Telex : 4206 DH
Office : 4771690 / 4771695 Cable : "NAVIGATION" Doha
Fax : 4771687 Bank A/c:QNB Main Branch
P.O.Box 1000, Doha - Qatar
Swift Code : QNBAQAQA
QAR A/C No. 0013-000309-060
USD A/C No. 0013-000309-061
I N V O I C E
Page: 1 /1
103519
INVOICE NO: SFWK/INV/012491 /2020
-1
MINISTRY OF MUNICIPALITY &
ENVIRONMENT DATE: 30/11/2020
DIRECTOR OF LAND & SURVEY DEPARTMENT,WEST BAY
EXPENSE A/C:
WEST BAY, DOHA -QATAR Doha QA 22423
W/O NO: 20-010907 / 1
W/O REF: MDO 20218
Vessel Name/Description : SURVEY BOAT "AL MASSAH"
(Commitment no.C2018/12)
Job No Description Line Amount
1100 BERTH PREPARATION, DOCKING AND 5,700.00
UNDOCKING:-
Berth prepared, vessel docked and later undocked on
completion of works.
As per our quotation EM 15489 Revised , item 1.B
1101 DRY DOCK HIRE:- 26,350.00
Dock hired from 12.11.2020 to 28.11.2020.
QRs 1,550 per day x 17 days = QRs 26,350.00
As per our quotation EM 15489 Revised , item 1.C
11 .
52 ANNUAL

In [14]:
draft = ""
cus_no = ""
cus_name = ""
inv_no = ""
date = ""
vessel_name = ""
inv_amount = ""

# Check if draft
if re.match(r"DRAFT\s+INVOICE", text):
    draft = "DRAFT"
elif re.match(r"PROFORMA\s+INVOICE", text):
    draft = "PROFORMA"
else:
    draft = "NORMAL"


inv_no = re.findall(r"[A-Z]{4}-[A-Z]{3}-[0-9]{6}|[A-Z]{4}/[A-Z]{3}/[0-9]{6}\s*/[0-9]{4}", text)[0]

date = re.findall(r"[0-9]{2}/[0-9]{2}/[0-9]{4}", text)[0]

for line in text.split("\n"):
    if re.match(r"INVOICE\s*NO\s*:", line):
        cus_no = line.split()[0]
    if re.match(r"DATE\s*:", line):
        cus_name = line.split()[0]

inv_amount_str = re.findall(r"Invoice\sAmount\s-->\s*([\d,]+\.\d+)?", text)[0]
inv_amount = re.sub(r',', '', inv_amount_str)


cus_no = re.findall(r"Page:\s*\d\s*/\s*\d+\n(\d{4,6})", text, flags=re.DOTALL)[0]
cus_name = re.findall(r"INVOICE\s*NO\s*:\s\S+\n(.*?)DATE\s*:", text, flags=re.DOTALL)[0]

vessel_name = re.findall(r"Vessel\s*Name/Description\s*:\s*(.*)", text)[0]

IndexError: list index out of range

In [308]:
[cus_no, cus_name, vessel_name, date, inv_no, inv_amount, draft, file]

['101138',
 'BOURBON GULF WLL',
 'M.V. “SURFER 1946”',
 '31/03/2021',
 'SFWK/INV/012671/2021',
 '8020.00',
 'NORMAL',
 'sfwk-inv-012893-2021.pdf']

# Tesseract

## Defining files that need OCR

In [18]:
# df = pd.read_excel("DataFrame1.xlsx")

In [32]:
files_for_ocr = df[df["Invoice number"].isna()]["File name"].to_list()

## Defining functions to convert pdf to image

In [2]:
import fitz
from PIL import Image
import pytesseract

In [3]:
file = "Draft  inv no 1.pdf"

In [4]:
def pdf_to_images(pdf_path):
    pdf_document = fitz.open(pdf_path)
    images = []

    for page_number in range(pdf_document.page_count):
        page = pdf_document.load_page(page_number)
        image = page.get_pixmap(matrix=fitz.Matrix(300 / 72, 300 / 72))
        images.append(image)

    return images

def perform_ocr(image):
    text = pytesseract.image_to_string(image)
    return text

Use this code to turn pdf into images

>images = pdf_to_images(file)  
>text = ""  
>for image in images:  
>&emsp;text += perform_ocr(Image.frombytes("RGB", (image.width, image.height), image.samples))  


## Performing OCR test

In [7]:
images = pdf_to_images("Invoices/"+file)
text = ""
for image in images:
    text += perform_ocr(Image.frombytes("RGB", (image.width, image.height), image.samples)) 

In [9]:
print(text)

Telephones

Office : 4771690 / 4771695 Cable : "NAVIGATION"
Fax

QATAR NAVIGATION (QPSC)
SHIP REPAIR & FABRICATION

Telex : 4206 DH

Doha
4771687 Bank A/c: QNB Main Branch

P.O.Box 1000, Doha - Qatar

Swift Code : QNBAQAQA

QAR A/C No. 0013-000309-060
USD A/C No. 0013-000309-061

DRAFT INVOICE

| 100936 a INVOICE NO : SFWK-DMY-007978
QATAR STEEL CO.LTD. DATE : 30/03/2021
Doha, Qatar DOHA QA 50090 W/O Number : 21-011045
W/O Ref : NMO21021

[_

EXPENSE A/C :

_

Vessel Name/Description: OPENING/CLOSING ACTIVITIES

Job No

1800

6000

for QATAR NAVIGATION (Q.S.C)

Description

Contract ref: 260636/23

GROUP#1: OPENING & CLOSING OF THE PLANT VESSELS AND DUCTS
HATCHES AND MANHOLES:-

Group I opening and closing of the plant vessels and ducts
hatches and manhole in furnace area group-1 carried out.

As per Addendum 1
GROUP # 3: OPENING & CLOSING OF ALL 3 NOS. DUST COLLECTION
SYSTEM: -

Carried out group III opening and closing of dust collection
systems as per group III work scope.

As per A

## Performing OCR

In [35]:
dd = list()

In [37]:
for file in files_for_ocr:
    if not file.endswith(".pdf"):
        continue
    
    cus_no = ""
    cus_name = ""
    text = ""
    inv_no = ""
    date = ""
    vessel_name = ""
    inv_amount = ""
    draft = ""
    
    #GET FULL TEXT OF INVOICE
    images = pdf_to_images("Invoices/"+file)
    for image in images:
        text += perform_ocr(Image.frombytes("RGB", (image.width, image.height), image.samples)) 
    
    if len(text)<200:
        dd.append(["", "", "", "", "", "", "", file])
        continue
            
    print(file)
    
    
    if re.match(r"DRAFT\s+INVOICE", text):
        draft = "DRAFT"
    elif re.match(r"PROFORMA\s+INVOICE", text):
        draft = "PROFORMA"
    else:
        draft = "NORMAL"

    try:
        inv_no = re.findall(r"[A-Z]{4}-[A-Z]{3}-[0-9]{6}|[A-Z]{4}/[A-Z]{3}/[0-9]{6}\s*/[0-9]{4}", text)[0]

        date = re.findall(r"[0-9]{2}/[0-9]{2}/[0-9]{4}", text)[0]

        for line in text.split("\n"):
            if re.match(r"INVOICE\s*NO\s*:", line):
                cus_no = line.split()[0]
            if re.match(r"DATE\s*:", line):
                cus_name = line.split()[0]

        inv_amount_str = re.findall(r"Invoice\sAmount\s-->\s*([\d,]+\.\d+)?", text)[0]
        inv_amount = re.sub(r',', '', inv_amount_str)


        cus_no = re.findall(r"Page:\s*\d\s*/\s*\d+\n(\d{4,6})", text, flags=re.DOTALL)[0]
        cus_name = re.findall(r"INVOICE\s*NO\s*:\s\S+\n(.*?)DATE\s*:", text, flags=re.DOTALL)[0]

        vessel_name = re.findall(r"Vessel\s*Name/Description\s*:\s*(.*)", text)[0]
    except:
        pass
    
    # print(files.index(file),"/"len(files), inv_no if inv_no else "lost", "===================================")
    
    dd.append([cus_no, cus_name, vessel_name, date, inv_no, inv_amount, draft, file])

21-011216-Indus Brizo.pdf
Draft  inv no 1.pdf
sfwk-inv-012586-2021.pdf
sfwk-inv-012587-2021.pdf
sfwk-inv-012588-2021.pdf
sfwk-inv-012666-2021.pdf
sfwk-inv-012699-2021.pdf
sfwk-inv-012771-2021.pdf
sfwk-inv-012772-2021.pdf
sfwk-inv-012773-2021.pdf
sfwk-inv-012837-1-2021-YXT One.pdf
sfwk-inv-012837-2-2021-YXT One.pdf
sfwk-inv-012920-1-2021- SPM Buoy- Qatar Petroleum.pdf
sfwk-inv-012920-1-2021.pdf
sfwk-inv-012920-2-2022- SPM Buoy at Mesaieed - Invoice.pdf
sfwk-inv-012920-2-2022.pdf
SFWK-INV-012920-2021-SPM BUOY-QATAR PETROLEUM.pdf
sfwk-inv-012921-2021.pdf
sfwk-inv-012968-2021.pdf
sfwk-inv-013071-2021-QEWC.pdf


In [38]:
df_ocred = pd.DataFrame(dd, columns=["Customer Number", "Customer Name", "Vessel Name", "Date", "Invoice number", "Invoice Amount", "Type", "File name"])

In [39]:
df_ocred

,Customer Number,Customer Name,Vessel Name,Date,Invoice number,Invoice Amount,Type,File name
0,,,,,,,,21-011216-Indus Brizo.pdf
1,,,,,,,,Draft inv no 1.pdf
2,,,,,,,,sfwk-inv-012586-2021.pdf
3,,,,,,,,sfwk-inv-012587-2021.pdf
4,,,,,,,,sfwk-inv-012588-2021.pdf
5,,,,,,,,sfwk-inv-012666-2021.pdf
6,,,,,,,,sfwk-inv-012699-2021.pdf
7,,,,,,,,sfwk-inv-012771-2021.pdf
8,,,,,,,,sfwk-inv-012772-2021.pdf
9,,,,,,,,sfwk-inv-012773-2021.pdf
